In [1]:
from get_financial import *

In [2]:
gicode = '005380'
get_html(gicode)
# get_stock_items(gicode)

005380_현대차_20230612.html file has been created.
현대차:주식 재무 분석 & 적정주가 계산_2023/06/12


'005380_현대차_20230612.html'